In [ ]:
# import numpy as np
# import plotly.graph_objects as go
# from itertools import product

# # ========================
# # Parâmetros geométricos
# # ========================
# Aresta_A = 5
# Altura   = 8

# DIM_ARESTAS = {"largura": 6}  # grossura das arestas
# DIM_ESFERAS = 6               # tamanho das esferas

# # ========================
# # Funções auxiliares
# # ========================

# def ponto_medio(p1, p2):
#     return tuple((np.array(p1) + np.array(p2)) / 2)

# def poligono_faces(points, faces):
#     """Gera as faces triangulares para plotagem no Mesh3d"""
#     x, y, z = zip(*points)
#     i, j, k = zip(*faces)
#     return go.Mesh3d(
#         x=x, y=y, z=z,
#         i=i, j=j, k=k,
#         opacity=0.25,
#         color="cyan",
#         flatshading=True,
#         name="Célula Primitiva",
#         visible=False,
#         showscale=False
#     )

# # ========================
# # Pontos do hexágono
# # ========================
# theta = np.linspace(0, 2*np.pi, 7)[:-1]
# verts_base = [(Aresta_A*np.cos(t), Aresta_A*np.sin(t), 0) for t in theta]
# verts_top  = [(x, y, Altura) for (x, y, z) in verts_base]

# # Nomeando
# A, B, C, D, E, F = verts_base
# Ap, Bp, Cp, Dp, Ep, Fp = verts_top
# R  = (0,0,0)
# Rp = (0,0,Altura)

# # Centro entre R e R'
# mid_R_Rp = ponto_medio(R, Rp)

# # ========================
# # Objetos gráficos
# # ========================
# fig = go.Figure()

# # Átomos (esferas)
# for name, coord in zip(["A","B","C","D","E","F","R","A'","B'","C'","D'","E'","F'","R'"],
#                        [A,B,C,D,E,F,R,Ap,Bp,Cp,Dp,Ep,Fp,Rp]):
#     fig.add_trace(go.Scatter3d(
#         x=[coord[0]], y=[coord[1]], z=[coord[2]],
#         mode="markers+text",
#         marker=dict(size=DIM_ESFERAS, color="blue"),
#         text=[name], textposition="top center",
#         name=name
#     ))

# # Arestas bases + conexões
# arestas = []
# arestas += [(A,B),(B,C),(C,D),(D,E),(E,F),(F,A)]   # base inferior
# arestas += [(Ap,Bp),(Bp,Cp),(Cp,Dp),(Dp,Ep),(Ep,Fp),(Fp,Ap)] # base superior
# arestas += [(v, vp) for v, vp in zip([A,B,C,D,E,F,R],[Ap,Bp,Cp,Dp,Ep,Fp,Rp])]  # verticais
# arestas += [(R,v) for v in [A,B,C,D,E,F]]  # centro da base inferior
# arestas += [(Rp,v) for v in [Ap,Bp,Cp,Dp,Ep,Fp]] # centro da base superior

# for p1, p2 in arestas:
#     fig.add_trace(go.Scatter3d(
#         x=[p1[0], p2[0]], y=[p1[1], p2[1]], z=[p1[2], p2[2]],
#         mode="lines",
#         line=dict(width=DIM_ARESTAS["largura"], color="black"),
#         showlegend=False
#     ))

# # ========================
# # Planos
# # ========================
# def plano_mesh(p1, p2, p3, color, name):
#     v1 = np.array(p2)-np.array(p1)
#     v2 = np.array(p3)-np.array(p1)
#     normal = np.cross(v1, v2)
#     d = -np.dot(normal, p1)

#     xs = np.linspace(-Aresta_A, Aresta_A, 10)
#     ys = np.linspace(-Aresta_A, Aresta_A, 10)
#     X, Y = np.meshgrid(xs, ys)
#     Z = (-normal[0]*X - normal[1]*Y - d)/normal[2]

#     # limitar ao volume
#     Z = np.where((Z>=0)&(Z<=Altura), Z, np.nan)

#     return go.Surface(
#         x=X, y=Y, z=Z,
#         opacity=0.5,
#         colorscale=[[0,color],[1,color]],
#         showscale=False,
#         visible=False,
#         name=name
#     )

# # Plano 1: A, C, R'
# fig.add_trace(plano_mesh(A, C, Rp, "red", "Plano 1"))

# # Plano 2: A, C, (R+R')/2
# fig.add_trace(plano_mesh(A, C, mid_R_Rp, "green", "Plano 2"))

# # ========================
# # Célula primitiva (paralelepípedo)
# # ========================
# # Pontos: A,B,C,R, Ap,Bp,Cp,Rp
# points_cell = [A,B,C,R, Ap,Bp,Cp,Rp]
# faces_cell = [
#     (0,1,2),(0,2,3),  # base inferior
#     (4,5,6),(4,6,7),  # base superior
#     (0,1,5),(0,5,4),  # lateral AB
#     (1,2,6),(1,6,5),  # lateral BC
#     (2,3,7),(2,7,6),  # lateral CR
#     (3,0,4),(3,4,7)   # lateral RA
# ]
# fig.add_trace(poligono_faces(points_cell, faces_cell))

# # ========================
# # Layout
# # ========================
# fig.update_layout(
#     scene=dict(
#         xaxis=dict(visible=True, tickvals=[0], ticktext=["0"]),
#         yaxis=dict(visible=True, tickvals=[0], ticktext=["0"]),
#         zaxis=dict(visible=True, tickvals=[0], ticktext=["0"]),
#         aspectmode="data"
#     ),
#     updatemenus=[
#         dict(
#             type="buttons",
#             direction="down",
#             buttons=list([
#                 # dict(label="Mostrar tudo", method="update",
#                 #      args=[{"visible":[True]*len(fig.data)}]),
#                 # dict(label="Somente Hexágono", method="update",
#                 #      args=[{"visible":[True if i < len(fig.data)-3 else False
#                 #                         for i in range(len(fig.data))]}]),
#                 # dict(label="Mostrar Plano 1", method="update",
#                 #      args=[{"visible":[True if (i < len(fig.data)-3) or i==len(fig.data)-3 else False
#                 #                         for i in range(len(fig.data))]}]),
#                 # dict(label="Mostrar Plano 2", method="update",
#                 #      args=[{"visible":[True if (i < len(fig.data)-3) or i==len(fig.data)-2 else False
#                 #                         for i in range(len(fig.data))]}]),
#                 dict(label="Mostrar Célula Primitiva", method="update",
#                      args=[{"visible":[True if (i < len(fig.data)-3) or i==len(fig.data)-1 else False
#                                         for i in range(len(fig.data))]}]),
#             ]),
#             showactive=True
#         )
#     ]
# )

# fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Configura para abrir no navegador
pio.renderers.default = "browser"

# ==============================
# Parâmetros
# ==============================
Aresta_A = 5          # comprimento da aresta do hexágono
altura = 8            # distância entre base inferior e superior

DIM_ESFERAS = 10
DIM_ARESTAS = {
    "espessura": 6,
    "cor": "black"
}

# ==============================
# Funções auxiliares
# ==============================
def hexagon_vertices(R, z=0):
    """Gera os 6 vértices de um hexágono regular no plano XY, com raio R e altura z."""
    vertices = []
    for k in range(6):
        angle = np.deg2rad(60 * k)  # 0, 60, 120...
        x = R * np.cos(angle)
        y = R * np.sin(angle)
        vertices.append([x, y, z])
    return np.array(vertices)

def make_triangle(p1, p2, p3, color="rgba(0,200,200,0.4)"):
    """Cria um triângulo no espaço definido por 3 pontos."""
    x, y, z = zip(p1, p2, p3)
    return go.Mesh3d(
        x=x, y=y, z=z,
        i=[0], j=[1], k=[2],  # apenas 1 triângulo
        color=color,
        opacity=0.5,
        name="Plano",
        visible=False
    )

# ==============================
# Construção da estrutura
# ==============================
R = Aresta_A

# Bases
base_inf = hexagon_vertices(R, z=0)
centro_inf = np.array([[0, 0, 0]])
base_sup = hexagon_vertices(R, z=altura)
centro_sup = np.array([[0, 0, altura]])

# Translada para que R esteja na origem
shift = centro_inf[0]
base_inf -= shift
base_sup -= shift
centro_inf -= shift
centro_sup -= shift

# Nomeação
labels_inf = ["A", "B", "C", "D", "E", "F"]
labels_sup = ["A'", "B'", "C'", "D'", "E'", "F'"]
labels_atoms = labels_inf + ["R"] + labels_sup + ["R'"]

atoms = np.vstack([base_inf, centro_inf, base_sup, centro_sup])

# ==============================
# Identificação dos pontos
# ==============================
# A = base_inf[0], C = base_inf[2]
A = base_inf[0]
C = base_inf[2]
R_ = centro_sup[0]
R = centro_inf[0]
M = (R + R_) / 2  # ponto médio

# ==============================
# Ligações
# ==============================
x_lines, y_lines, z_lines = [], [], []

# Lados
for base in [base_inf, base_sup]:
    for i in range(6):
        j = (i + 1) % 6
        x_lines += [base[i,0], base[j,0], None]
        y_lines += [base[i,1], base[j,1], None]
        z_lines += [base[i,2], base[j,2], None]

# Vértices ↔ centro
for v in base_inf:
    x_lines += [v[0], R[0], None]
    y_lines += [v[1], R[1], None]
    z_lines += [v[2], R[2], None]

for v in base_sup:
    x_lines += [v[0], R_[0], None]
    y_lines += [v[1], R_[1], None]
    z_lines += [v[2], R_[2], None]

# Verticais
for v_inf, v_sup in zip(base_inf, base_sup):
    x_lines += [v_inf[0], v_sup[0], None]
    y_lines += [v_inf[1], v_sup[1], None]
    z_lines += [v_inf[2], v_sup[2], None]

# ==============================
# Plotly - Átomos e Ligações
# ==============================
trace_atoms = go.Scatter3d(
    x=atoms[:,0], y=atoms[:,1], z=atoms[:,2],
    mode='markers',
    marker=dict(size=DIM_ESFERAS, color='red', opacity=0.9),
    name='Átomos'
)

trace_labels = go.Scatter3d(
    x=atoms[:,0], y=atoms[:,1], z=atoms[:,2],
    mode='text',
    text=labels_atoms,
    textposition="top center",
    textfont=dict(size=14, color="blue"),
    name="Labels"
)

trace_bonds = go.Scatter3d(
    x=x_lines, y=y_lines, z=z_lines,
    mode='lines',
    line=dict(width=DIM_ARESTAS["espessura"], color=DIM_ARESTAS["cor"]),
    name='Ligações'
)

# ==============================
# Planos (triângulos)
# ==============================
plano1 = make_triangle(A, C, R_, color="rgba(0,255,0,0.4)")
plano2 = make_triangle(A, C, M, color="rgba(0,0,255,0.4)")

# ==============================
# Layout e Botões
# ==============================
layout = go.Layout(
    title='Rede Cristalina Hexagonal Simples (HS)',
    scene=dict(
        xaxis=dict(title='X', tickvals=[0], showgrid=True),
        yaxis=dict(title='Y', tickvals=[0], showgrid=True),
        zaxis=dict(title='Z', tickvals=[0], showgrid=True),
        aspectmode='data'
    ),
    updatemenus=[dict(
        type="buttons",
        showactive=True,
        buttons=[
            dict(label="Mostrar Plano 1", method="update",
                 args=[{"visible": [True, True, True, True, False]}]),
            dict(label="Mostrar Plano 2", method="update",
                 args=[{"visible": [True, True, True, False, True]}]),
            dict(label="Ocultar Planos", method="update",
                 args=[{"visible": [True, True, True, False, False]}]),
        ]
    )]
)

# Ordem: bonds, atoms, labels, plano1, plano2
fig = go.Figure(data=[trace_bonds, trace_atoms, trace_labels, plano1, plano2], layout=layout)

fig.show()
